# Neyer testing notes
look in scipy\misc\ring down tests for data:  

write some python code to analyze the data and explore the calculations

copy the cells below to a new file called neyer?  get the name for this analysis, maybe binomial, experimental trials pass fail analysis

Neyer D-Optimal Test
--------------------
The Neyer D-Optimal Test is a sensitivity test. It can be used to answer questions such as "How far can a carton of eggs fall, on average, before one breaks?" If these egg cartons are very expensive, the person running the test would like to minimize the number of cartons dropped, to keep the experiment cheaper and to perform it faster. The Neyer test allows the experimenter to choose the experiment that gives the most information. In this case, given the history of egg cartons which have already been dropped, and whether those cartons broke or not, the Neyer test says "you will learn the most if you drop the next egg carton from a height of 32.123 meters."

The Neyer-d optimal test was described by Barry T. Neyer in 1994. This method has replaced the earlier Bruceton analysis or "Up and Down Test" that was devised by Dixon and Mood in 1948 to allow computation with pencil and paper. Samples are tested at various stimulus levels, and the results (response or no response) noted. The Neyer Test guides the experimenter to pick test levels that provide the maximum amount of information. Unlike previous methods that have been developed, this method requires the use of a computer program to calculate the test levels.

Although not directly related to the test method, the likelihood ratio analysis method is often used to analyze the results of tests conducted with the Neyer D-Optimal test. The combined test and analysis methods are commonly known as the Neyer Test.


In the design of experiments for estimating statistical models, optimal designs allow parameters to be estimated without bias and with minimum variance. A non-optimal design requires a greater number of experimental runs to estimate the parameters with the same precision as an optimal design. In practical terms, optimal experiments can reduce the costs of experimentation.

The optimality of a design depends on the statistical model and is assessed with respect to a statistical criterion, which is related to the variance-matrix of the estimator. Specifying an appropriate model and specifying a suitable criterion function both require understanding of statistical theory and practical knowledge with designing experiments.

D-optimality (determinant)
A popular criterion is D-optimality, which seeks to minimize |(X'X)−1|, or equivalently maximize the determinant of the information matrix X'X of the design. This criterion results in maximizing the differential Shannon information content of the parameter estimates.

references

http://neyersoftware.com/Papers/Analysis/SensitivityAnalysis.htm

http://www.oecd.org/chemicalsafety/testing/1836220.pdf

https://www.ida.org/idamedia/Corporate/Files/Publications/IDA_Documents/OED/2016/D-5829.pdf


In [1]:
import numpy as np
import pandas as pd

This is the report from the neyer software Table 5
```
       Stimulus   Failures  Successes

         1075,        1,        0
         1123,        1,        0
         1126,        1,        0
         1128,        1,        0
         1131,        1,        0
         1133,        1,        0
         1134,        1,        0
         1136,        1,        0
         1138,        1,        1    <- Lowest success
         1142,        1,        1
         1148,        1,        0
         1150,        1,        0
         1154,        0,        1
         1159,        0,        1
         1169,        1,        0
         1178,        1,        0
         1182,        0,        1
         1185,        0,        1
         1186,        1,        0
         1189,        0,        2
         1193,        0,        2
         1194,        1,        0    <- Highest failure
         1195,        0,        1
         1198,        0,        1
         1200,        0,        1
         1202,        0,        1

Number of stimulus values = 26, Number of tests = 30
Mu = 1165.2 (volts)                Sigma = 30.1 (volts)
0.0010 Level = 1072.1 (volts)   0.9990 Level = 1258.3 (volts)

Estimated Lower Bound of Variance of Mu = 65.4 (volts^2)
Estimated Lower Bound of Variance of Sigma = 90.0 (volts^2)
Estimated CoVariance of Mu and Sigma = 3.16 (volts^2)

The log of the likelihood function = -14.363
The maximum bounded individual confidence = 1.00
The maximum bounded joint confidence = 0.998

Calculated confidence levels:

0.950 One Sided Individual Confidence 0.001  No-Fire Level = 980.2
0.950 One Sided Individual Confidence 0.999 All-Fire Level = 1353.3
```

In [21]:
# build the data frame, data was expanded for multiple success, need to check
#Test1 is from the orginal data from table 5, Test1a has the multiple successes expanded
# these still need fixing
TEST1 = {
    'Stimulus' :[1075,1123,1126,1128,1131,1133,1134,1136,1138,1142,1148,1150,1154,1159,
                 1169,1178,1182,1185,1186,1189,1193,1194,1195,1198,1200,1202],
    'Failures' : [1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0],
    'Successes' :[0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,1,0,2,2,0,1,1,1,1]
    }

TEST1a = {
    'Stimulus' :[1075,1123,1126,1128,1131,1133,1134,1136,1138,1138,1142,1142,1148,1150,1154,1159,
                 1169,1178,1182,1185,1186,1189,1189,1193,1193,1194,1195,1198,1200,1202],
    'Failures' : [1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0],
    'Successes' :[0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,1,1]
    }
df1 = pd.DataFrame(TEST1a)

In [25]:
# number of tests & values
Number_of_tests = df1['Failures'].sum() + df1['Successes'].sum()
print('Number of tests = {:d}'.format(Number_of_tests))
Number_of_stimulus_values = df1['Stimulus'].count()
print('Number of stimulus values = {:d}'.format(Number_of_stimulus_values))

Number_of_successes = df1['Successes'].sum()
print('Number of successes = {:d}'.format(Number_of_successes))
Number_of_failures = df1['Failures'].sum()
print('Number of failures = {:d}'.format(Number_of_failures))

Number of tests = 31
Number of stimulus values = 30
Number of successes = 15
Number of failures = 16


##### looking at data in terms of a normal distribution

In [23]:
# calculate the mu = mean & sigma = standard deviation, what other type of dispributions are supported?
# expand the stimulus list, for now just operate on the column
df1['Stimulus'].mean()

1160.3333333333333

In [18]:
df1['Stimulus'].std()

32.10787043569449

##### Binomial distribution
In probability theory and statistics, the binomial distribution with parameters n and p is the discrete probability distribution of the number of successes in a sequence of n independent experiments, each asking a yes–no question, and each with its own boolean-valued outcome: a random variable containing single bit of information: success/yes/true/one (with probability p) or failure/no/false/zero (with probability q = 1 − p).

https://en.wikipedia.org/wiki/Binomial_distribution

n is the number of tests

p is the probability, maybe 0.5, since the stimulus is adjusted to give equal successes and failures?

In [0]:
n = Number_of_tests #
k = Number_of_successes

In [0]:
# read thru each row and append to a list the stimulus value


Neyer
// Neyer Test Method
// Jan 3, 2011
// Version 0

function p = find_p(n)
  // (1) start at level (n)
  // (2) find previous trial(p) so that there are the same number
  // of successes as failures between trials(n) through (p)
  // returns p = trial where there are equal successes as failures
  // returns p = 0 if cannot find p
  
  // look at cases where n equals 1 or 2
  select n
    case 1 then
      p = 0;
    case 2 then
      s = sum(R(1:2));  // can be 0, 1, 2
      if s == 1 then
        p = 1;
      else
        p = 0;
      end;  
    else

      // for all n > 2, do the following
      p = n-1;  // start search at previous trial
      while p > 0
        s = sum(R(p:n));  // add up the number of success
        // if sum == to search back depth/2 then return p
        if ((n-p) == 1) & (s == 1) then
          break;
        end;
        k = size(R(p:n));
        if s == (k(1)/2) then
          break;
        else
          p = p-1;  //look back farther
        end;  
      end;  

  end;
endfunction;

// Input Initial Conditions
LL = input("Input Lower Limit: ");  // Lower limit
UL = input("Input Upper Limit: ");  // Upper limit
num_tests = input("Input Number of Tests: "); // number of test in series
m_est = input("Input estimated mean: "); // estimated mean
sd_est = input("Input estimated standard deviation: "); // estimated standard deivation

L(1) = round(LL + (UL-LL)/2);  // starting level

// Neyer procedure
// first time?
Next = (MuMin+MuMax)/2;

MaxS = Max(Stimuli);
MinS = min(Stimuli);

// all failures?
Next = maximum of 
(MuMax+MaxS)/2,
MaxS+2*SigmaG,
2*MaxS-MinS

// all failures
Next = minimum of
(Mumin+MinS)/2
MinS-2*SigmaG
2*MinS-MaxS

MinX = min(Success)
Max0 = max(Failure)
Diff = MinX-Max0

// is Diff > SigmaG ?
Next = (MuMin+MuMax)/2

// Comput MLE of Mu & Sigma

//Sigma == 0 ?
Mu = (minX+Max0)/2
Sigma = SigmaG
SigmaG = 0.8*SigmaG

Mu = maximum of
MinS
min(Mu,MaxS)
Sigma = min(Sigma,MaxS-MinS)

// Next = level which maximuzes information matrix




//
//  old code
//


// main test loop 
for n = 1:num_tests
  printf("Test #%d, Stimulus Level = %d,  ",n,round(L(n)));

  // Test unit
  result = input("What was the result? (success=1, failure=0): ");

  // Store result
  R(n) = result;

  // Find next level
  p = find_p(n);
  printf("n=%d, p=%d\n\r",n,p);
  if p > 0
    printf("L(n)=%.2f, L(p)=%.2f\n\r",L(n),L(p));
  end;
  if p > 0 then
    if L(n) > L(p) then
      L(n+1) = L(p) + (L(n) - L(p))/2;
    else
      L(n+1) = L(n) + (L(p)-L(n))/2;
    end;
  else;
    // cannot find (p)
    if R(n) == 0 then 
      L(n+1) = LL + (L(n)-LL)/2;  //new level if last test was failure
    else;
      L(n+1) = L(n) + (UL - L(n))/2; //new level if last test was a success
    end;
  end;
end;

// compute statistics
m = mean(L);
printf("mean stimulus level = %.2f\n\r",m);
sd = st_deviation(L);
printf("Standard deviation of stimulus level = %.2f\n\r",sd);

// plots
histplot(10,L);
scf(1);
plot(L,"*");

//end

